In [ ]:
# | default_exp application

In [ ]:
# | export

from typing import *
from typing import get_type_hints

from enum import Enum
from pathlib import Path
import json
import yaml
from copy import deepcopy
from os import environ
from datetime import datetime, timedelta
import tempfile
from contextlib import contextmanager, asynccontextmanager
import time
from inspect import signature

from fastcore.foundation import patch
from fastcore.meta import delegates

import anyio
import asyncio
from asyncio import iscoroutinefunction  # do not use the version from inspect
import httpx
from fastapi import FastAPI
from fastapi import status, Depends, HTTPException, Request, Response
from fastapi.openapi.docs import get_swagger_ui_html, get_redoc_html
from fastapi.openapi.utils import get_openapi
from fastapi.responses import FileResponse, RedirectResponse
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from pydantic import Field, HttpUrl, EmailStr, PositiveInt
from pydantic.schema import schema
from pydantic.json import timedelta_isoformat
from aiokafka import AIOKafkaProducer

import confluent_kafka
from confluent_kafka import Producer
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Message, KafkaError
import asyncer

import fast_kafka_api.logger
fast_kafka_api.logger.should_supress_timestamps = True

import fast_kafka_api
from fast_kafka_api._components.aiokafka_loop import aiokafka_consumer_loop
from fast_kafka_api.confluent_kafka import AIOProducer
from fast_kafka_api.confluent_kafka import create_missing_topics
from fast_kafka_api.asyncapi import (
    KafkaMessage,
    export_async_spec,
    _get_msg_cls_for_method,
)
from fast_kafka_api.asyncapi import (
    KafkaBroker,
    ContactInfo,
    KafkaServiceInfo,
    KafkaBrokers,
)
from fast_kafka_api.logger import get_logger
from fast_kafka_api.testing import true_after

In [ ]:
# | export
logger = get_logger(__name__)

In [ ]:
# | export
logger = get_logger(__name__, level=0)

In [ ]:
logger.debug("ok")

In [ ]:
import pytest
import yaml
import unittest.mock
from dataclasses import dataclass

import nest_asyncio

import uvicorn
from fastapi.testclient import TestClient
from starlette.datastructures import Headers

from rich.pretty import pprint

from fast_kafka_api.confluent_kafka import create_testing_topic

In [ ]:
# | eval: false
# allows async calls in notebooks

nest_asyncio.apply()

In [ ]:
# | export
class KafkaErrorMsg(KafkaMessage):
    topic: str = Field(..., description="topic where exception occurred")
    raw_msg: Optional[bytes] = Field(None, description="raw message string")
    error: str = Field(..., description="exception triggered by the message")
        
ProduceCallable = Callable[[str, KafkaMessage], None]
TopicCallable = Callable[[KafkaMessage, ProduceCallable], None]

In [ ]:
# | export


def _get_topic_name(
    on_topic: TopicCallable, prefix: str = "on_"
) -> str:
    topic = on_topic.__name__
    if not topic.startswith(prefix) or len(topic) <= len(prefix):
        raise ValueError(f"Function name '{topic}' must start with {prefix}")
    topic = topic[len(prefix) :]

    return topic

In [ ]:
def on_topic_name_1():
    pass


assert _get_topic_name(on_topic_name_1) == "topic_name_1"

assert _get_topic_name(on_topic_name_1, prefix="on_topic_") == "name_1"

In [ ]:
# | export


class FastKafkaAPI(FastAPI):
    def __init__(
        self,
        *,
        title: str = "FastKafkaAPI",
        contact: Optional[Dict[str, Union[str, Any]]] = None,
        kafka_brokers: Optional[Dict[str, Any]] = None,
        kafka_config: Dict[str, Any],
        root_path: Optional[Union[Path, str]] = None,
        num_partitions: Optional[int] = None,
        replication_factor: Optional[int] = None,
        **kwargs,
    ):
        self._kafka_config = kafka_config
        self.num_partitions = num_partitions
        self.replication_factor = replication_factor

        if root_path is None:
            root_path = Path(".")
        self._root_path = Path(root_path)

        if kafka_brokers is None:
            kafka_brokers = {
                "localhost": KafkaBroker(
                    url="https://localhost",
                    description="Local (dev) Kafka broker",
                    port="9092",
                )
            }
        if contact is None:
            contact = dict(
                name="author", url="https://www.google.com", email="noreply@gmail.com"
            )

        super().__init__(title=title, contact=contact, **kwargs)

        self._store: Dict[str, Dict[str, Tuple[Union[TopicCallable, AIOKafkaProducer], Dict[str, Any]]],] = {
            "consumers": {},
            "producers": {},
        }
        self._on_error_topic: Optional[str] = None

        contact_info = ContactInfo(**contact)  # type: ignore
        self._kafka_service_info = KafkaServiceInfo(
            title=self.title,
            version=self.version,
            description=self.description,
            contact=contact_info,
        )
        self._kafka_brokers = KafkaBrokers(brokers=kafka_brokers)

        self._confluent_producer: Optional[AIOProducer] = None

        self._asyncapi_path = self._root_path / "asyncapi"
        (self._asyncapi_path / "docs").mkdir(exist_ok=True, parents=True)
        (self._asyncapi_path / "spec").mkdir(exist_ok=True, parents=True)
        self.mount(
            "/asyncapi",
            StaticFiles(directory=self._asyncapi_path / "docs"),
            name="asyncapi",
        )

        self._is_shutting_down: bool = False
        self._kafka_consumer_tasks: List[asyncio.Task[Any]] = []
        self._kafka_producer_tasks: List[asyncio.Task[Any]] = []

        @self.get("/", include_in_schema=False)
        def redirect_root_to_asyncapi():
            return RedirectResponse("/asyncapi")

        @self.get("/asyncapi", include_in_schema=False)
        async def redirect_asyncapi_docs():
            return RedirectResponse("/asyncapi/index.html")

        @self.get("/asyncapi.yml", include_in_schema=False)
        async def download_asyncapi_yml():
            return FileResponse(self._asyncapi_path / "spec" / "asyncapi.yml")

        @self.on_event("startup")
        async def __on_startup(app=self):
            app._on_startup()

        @self.on_event("shutdown")
        async def __on_shutdown(app=self):
            await app._on_shutdown()

    async def _on_startup(self) -> None:
        raise NotImplemented

    async def _on_shutdown(self) -> None:
        raise NotImplemented

    def _add_topic(
        self,
        *,
        store_key: str,
        topic: str,
        f: Callable[[KafkaMessage], Any],
        on_error: bool,
        **kwargs,
    ):
        """Stores function `f` under key `topic` in `store`

        Params:
            store_key: either `consumers` or `producers`
            topic: the name of the topic
            f: callback function called on receiving a message for consumers or on delivery report for producers
            on_error: True for at most one producer topic used for outputting errors

        Raises:
            ValueError:
                - if store_key not one of either `consumers` or `producers`,
                - if key `topic` is already in `self._store[store_key]`, or
                - if `on_error` is already set
        """
        raise NotImplementedError

    def _register_kafka_callback(
        self,
        *,
        store_key: str,
        topic: Optional[str] = None,
        prefix: str = "on_",
        on_error: bool = False,
        **kwargs,
    ) -> TopicCallable:
        """ """
        raise NotImplementedError

    def consumes(
        self,
        topic: Optional[str] = None,
        prefix: str = "on_",
        **kwargs,
    ) -> TopicCallable:
        """Decorator registering the callback called when a message is received in a topic.

        This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

        Params:
            topic: todo

        Returns:
            A function returning the same function

        """
        return self._register_kafka_callback(
            store_key="consumers", topic=topic, prefix=prefix, on_error=False, **kwargs
        )

    def produces(
        self,
        topic: Optional[str] = None,
        prefix: str = "on_",
        on_error: bool = False,
        **kwargs,
    ) -> Callable[[KafkaMessage], None]:
        """Decorator registering the callback called when delivery report for a produced message is received

        This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

        Params:
            topic: TODO

        Returns:
            A function returning the same function

        """
        return self._register_kafka_callback(
            store_key="producers",
            topic=topic,
            prefix=prefix,
            on_error=on_error,
            produces=True,
            **kwargs,
        )

    def produces_on_error(
        self,
        topic: Optional[str] = None,
        prefix: str = "on_",
        **kwargs,
    ) -> Callable[[KafkaMessage], None]:
        return self._register_kafka_callback(
            store_key="producers", topic=topic, prefix=prefix, on_error=True, **kwargs
        )

    def produce(
        self,
        topic: str,
        msg: KafkaMessage,
        on_delivery: Optional[Callable[[KafkaMessage, Message], None]] = None,
    ):
        return self.produce_raw(
            topic=topic, raw_msg=msg.json().encode("utf-8"), on_delivery=on_delivery
        )

    def produce_raw(
        self,
        topic: str,
        raw_msg: Union[str, bytes],
        on_delivery: Optional[Callable[[KafkaMessage, Message], None]] = None,
    ) -> "asyncio.Future[Any]":
        raise NotImplementedError

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
    "bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}",
    "group.id": f"{kafka_server_url}:{kafka_server_port}_group",  # ToDo: Figure out msg deletion from kafka after consuming once
    "auto.offset.reset": "earliest",
}


def create_testing_app():
    app = FastKafkaAPI(
        kafka_brokers={
            "local": {
                "url": "kafka",
                "name": "development",
                "description": "Local (dev) Kafka broker",
                "port": 9092,
            }
        },
        kafka_config=kafka_config,
        root_path="/tmp/000_FastKafkaAPI",
    )

    return app

In [ ]:
app = create_testing_app()

In [ ]:
# | export


@patch
def _add_topic(
    self: FastKafkaAPI,
    *,
    store_key: str,
    topic: str,
    f: Callable[[KafkaMessage], Any],
    on_error: bool = False,
    **kwargs
):
    if store_key not in ["consumers", "producers"]:
        raise ValueError(
            f"store_key must be one of 'consumers', 'producers', it is '{store_key}' instead"
        )
    if on_error:
        if store_key == "consumers":
            raise ValueError(
                "`on_error` can be true only when `store_key` is `producers`"
            )
        if self._on_error_topic is not None:
            raise ValueError(
                f"`on_error` must be unique, it is already set to '{self._on_error_topic}'"
            )
        self._on_error_topic = topic

    if topic in self._store["consumers"].keys():
        raise ValueError(f"Topic '{topic}' is already in consumers.")
    if topic in self._store["producers"].keys():
        raise ValueError(f"Topic '{topic}' is already in producers.")
    self._store[store_key][topic] = (f, kwargs)

In [ ]:
app = create_testing_app()


def f(KafkaMessage):
    pass


with pytest.raises(ValueError) as e:
    app._add_topic(store_key="random_thing", topic="topic_name_1", f=f, on_error=False)
assert (
    str(e.value)
    == "store_key must be one of 'consumers', 'producers', it is 'random_thing' instead"
), str(e.value)

with pytest.raises(ValueError) as e:
    app._add_topic(store_key="consumers", topic="topic_name_1", f=f, on_error=True)
assert (
    str(e.value) == "`on_error` can be true only when `store_key` is `producers`"
), str(e.value)

app._add_topic(store_key="producers", topic="topic_name_1", f=f, on_error=True)
assert app._store["producers"]["topic_name_1"] == (f, {})

with pytest.raises(ValueError) as e:
    app._add_topic(store_key="producers", topic="topic_name_2", f=f, on_error=True)
assert (
    str(e.value) == "`on_error` must be unique, it is already set to 'topic_name_1'"
), str(e.value)

app._add_topic(store_key="producers", topic="topic_name_2", f=f, on_error=False, servers="servers")
assert app._store["producers"]["topic_name_2"] == (f, dict(servers="servers"))

with pytest.raises(ValueError) as e:
    app._add_topic(store_key="producers", topic="topic_name_2", f=f, on_error=False)
assert str(e.value) == "Topic 'topic_name_2' is already in producers.", str(e.value)

with pytest.raises(ValueError) as e:
    app._add_topic(store_key="consumers", topic="topic_name_2", f=f, on_error=False)
assert str(e.value) == "Topic 'topic_name_2' is already in producers.", str(e.value)

app._add_topic(store_key="consumers", topic="topic_name_3", f=f, on_error=False, my_param=42)
assert app._store["consumers"]["topic_name_3"] == (f, {"my_param": 42})

In [ ]:
# | export
def _get_first_func_arg_type(f: Callable[[Any], Any]) -> Type[Any]:
    return list(get_type_hints(f).values())[0]

In [ ]:
def some_f(x: str):
    pass


actual = _get_first_func_arg_type(some_f)
assert actual == str

In [ ]:
def produce(self, topic, func):
    async def _produce(*args, **kwargs):
        return_val = func(*args, **kwargs)
        logger.debug(f"Producing msg {return_val}")
        logger.debug(f"{self._store}")
        producer, _ = self._store["producers"][topic]
        fut = await producer.send(topic, return_val.json().encode("utf-8"))
        msg = await fut
        return return_val
    return _produce

In [ ]:
# | export

@patch
def _register_kafka_callback(
    self: FastKafkaAPI,
    *,
    store_key: str,
    topic: Optional[str] = None,
    prefix: str = "on_",
    on_error: bool = False,
    produces: bool = False,
    **kwargs,
) -> Callable[[KafkaMessage, Callable[[str, KafkaMessage], None]], None]:
    """
    """

    def _decorator(
        on_topic: Callable[[KafkaMessage, Callable[[str, KafkaMessage], None]], None],
        store_key: str = store_key,
        topic: str = topic,
        on_error: bool = on_error,
        kwargs=kwargs
    ) -> Callable[[KafkaMessage, Callable[[str, KafkaMessage], None]], None]:
        if topic is None:
            topic = _get_topic_name(on_topic=on_topic, prefix=prefix)

        first_arg_type = _get_first_func_arg_type(on_topic)
        if on_error and first_arg_type != KafkaErrorMsg:
            raise ValueError(
                f"The first argument of a decorator handling errors must be KafkaErrorMsg, it is '{first_arg_type}' instead"
            )
        self._add_topic(
            store_key=store_key, topic=topic, f=on_topic, on_error=on_error, **kwargs
        )
        
        
        return produce(self, topic, on_topic) if produces else on_topic

    return _decorator

In [ ]:
app = create_testing_app()


@app._register_kafka_callback(store_key="consumers", server="server")
def on_topic_1(msg: KafkaMessage):
    pass
assert app._store["consumers"]["topic_1"] == (on_topic_1, {"server": "server"}), app._store

@app._register_kafka_callback(store_key="consumers", topic="topic_2")
def some_callback(msg: KafkaMessage):
    pass

assert app._store["consumers"]["topic_2"] == (some_callback, {}), app._store

In [ ]:
class MyInfo(KafkaMessage):
    mobile: str = Field(..., example="+385987654321")
    name: str = Field(..., example="James Bond")


class MyMsgUrl(KafkaMessage):
    info: MyInfo = Field(..., example=dict(mobile="+385987654321", name="James Bond"))
    url: HttpUrl = Field(..., example="https://sis.gov.uk/agents/007")


class MyMsgEmail(KafkaMessage):
    msg_url: MyMsgUrl = Field(
        ...,
        example=dict(
            info=dict(mobile="+385987654321", name="James Bond"),
            url="https://sis.gov.uk/agents/007",
        ),
    )
    email: EmailStr = Field(..., example="agent-007@sis.gov.uk")


def setup_testing_app():
    app = create_testing_app()

    @app.consumes("my_topic_1")
    def on_my_topic_one(msg: MyMsgUrl):
        logger.debug(f"on_my_topic_one(msg={msg},)")

    @app.consumes()
    async def on_my_topic_2(msg: MyMsgEmail):
        logger.debug(f"on_my_topic_2(msg={msg},)")

    with pytest.raises(ValueError) as e:

        @app.consumes()
        def my_topic_3(msg: MyMsgEmail):
            raise NotImplemented

    with pytest.raises(ValueError) as e:

        @app.produces()
        async def on_my_topic_1(msg: MyMsgUrl, kafka_msg: Message):
            pass

    @app.produces()
    def on_my_topic_3(msg: MyMsgUrl, kafka_msg: Message):
        logger.debug(f"on_my_topic_3(msg={msg},, kafka_msg={kafka_msg})")

    @app.produces()
    def on_my_topic_4(msg: MyMsgEmail, kafka_msg: Message):
        logger.debug(f"on_my_topic_4(msg={msg},, kafka_msg={kafka_msg})")

    @app.produces_on_error()
    async def on_my_topic_error(raw_msg: KafkaErrorMsg, kafka_err: KafkaError):
        logger.warning(f"on_error(raw_msg={raw_msg}, kafka_err={kafka_err},)")
        
    @app.produces()
    def on_my_topic_test(msg: MyMsgUrl):
            logger.debug(f"on_topic_test(msg={msg})")
            return msg

    return app, on_my_topic_test

In [ ]:
app, _= setup_testing_app()

pprint(app._store)
assert set(app._store["consumers"].keys()) == set(["my_topic_1", "my_topic_2"])
assert set(app._store["producers"].keys()) == set(
    ["my_topic_3", "my_topic_4", "my_topic_error", "my_topic_test"]
), set(app._store["producers"].keys())

print(f"app._kafka_service_info={app._kafka_service_info}")

print(f"app._kafka_brokers={app._kafka_brokers}")

In [ ]:
# | export


def populate_consumers(
    *,
    app: FastKafkaAPI,
    is_shutting_down_f: Callable[[], bool],
) -> List[asyncio.Task]:
    config: Dict[str, Any] = app._kafka_config
    tx = [
        asyncio.create_task(
            aiokafka_consumer_loop(
                topics=[topic],
#                 bootstrap_servers=configuration.get("bootstrap.servers",config["bootstrap.servers"]),
#                 auto_offset_reset=configuration.get("auto_offset_reset", config["auto_offset_reset"]),
#                 max_poll_records=configuration.get("max_poll_records", config["max_poll_records"]),
#                 max_buffer_size=configuration.get("max_buffer_size", config["max_buffer_size"]),
                bootstrap_servers=config["bootstrap.servers"],
                auto_offset_reset="earliest",
                max_poll_records=100,
                max_buffer_size=100,
                callbacks={topic: consumer},
                msg_types={topic: signature(consumer).parameters['msg'].annotation},
                is_shutting_down_f=is_shutting_down_f,
            )
        )
        for topic, (consumer, configuration) in app._store["consumers"].items()
    ]

    return tx

In [ ]:
def populate_producers(
    *,
    app: FastKafkaAPI
) -> None:
    config: Dict[str, Any] = app._kafka_config
    print(app._store["producers"])
    for topic, (_, configuration) in app._store["producers"].items():
        producer = AIOKafkaProducer(
            bootstrap_servers=config["bootstrap.servers"]
        )
        asyncio.create_task(producer.start())
        app._store["producers"][topic] = (producer, configuration)
    print(app._store["producers"])

In [ ]:
# | export


@patch
def _on_startup(self: FastKafkaAPI) -> None:
#     export_async_spec(
#         consumers=self._store["consumers"],  # type: ignore
#         producers=self._store["producers"],  # type: ignore
#         kafka_brokers=self._kafka_brokers,
#         kafka_service_info=self._kafka_service_info,
#         asyncapi_path=self._asyncapi_path,
#     )

    self._is_shutting_down = False

    def is_shutting_down_f(self: FastKafkaAPI = self) -> bool:
        return self._is_shutting_down

    self._kafka_consumer_tasks = populate_consumers(
        app=self,
        is_shutting_down_f=is_shutting_down_f,
    )
    
    populate_producers(app=self)

    self._confluent_producer = AIOProducer(self._kafka_config)
    logger.info("AIOProducer created.")


@patch
async def _on_shutdown(self: FastKafkaAPI) -> None:
    self._is_shutting_down = True
    await asyncio.wait(self._kafka_consumer_tasks)
    self._confluent_producer.close()  # type: ignore
    [await producer.stop() for producer, _ in self._store["producers"].values()]
    logger.info("AIOProducer closed.")

    self._is_shutting_down = False

In [ ]:
@asynccontextmanager
async def start_test_app():
    app, test_produce_function = setup_testing_app()
    try:
        app._on_startup()
        yield app, test_produce_function
    finally:
        await app._on_shutdown()

In [ ]:
expected = """asyncapi: 2.5.0
info:
  title: FastKafkaAPI
  version: 0.1.0
  description: ''
  contact:
    name: author
    url: https://www.google.com
    email: noreply@gmail.com
servers:
  local:
    url: kafka
    description: Local (dev) Kafka broker
    protocol: kafka
    variables:
      port:
        default: '9092'
channels:
  my_topic_1:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_2:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_3:
    publish:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_4:
    publish:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_error:
    publish:
      message:
        $ref: '#/components/messages/KafkaErrorMsg'
components:
  messages:
    MyMsgUrl:
      payload:
        title: MyMsgUrl
        type: object
        properties:
          info:
            title: Info
            example:
              mobile: '+385987654321'
              name: James Bond
            allOf:
            - $ref: '#/components/schemas/MyInfo'
          url:
            title: Url
            example: https://sis.gov.uk/agents/007
            minLength: 1
            maxLength: 2083
            format: uri
            type: string
        required:
        - info
        - url
        example:
          info:
            mobile: '+385987654321'
            name: James Bond
          url: https://sis.gov.uk/agents/007
    MyMsgEmail:
      payload:
        title: MyMsgEmail
        type: object
        properties:
          msg_url:
            title: Msg Url
            example:
              info:
                mobile: '+385987654321'
                name: James Bond
              url: https://sis.gov.uk/agents/007
            allOf:
            - $ref: '#/components/messages/MyMsgUrl'
          email:
            title: Email
            example: agent-007@sis.gov.uk
            type: string
            format: email
        required:
        - msg_url
        - email
        example:
          msg_url:
            info:
              mobile: '+385987654321'
              name: James Bond
            url: https://sis.gov.uk/agents/007
          email: agent-007@sis.gov.uk
    KafkaErrorMsg:
      payload:
        title: KafkaErrorMsg
        type: object
        properties:
          topic:
            title: Topic
            description: topic where exception occurred
            type: string
          raw_msg:
            title: Raw Msg
            description: raw message string
            format: binary
            type: string
          error:
            title: Error
            description: exception triggered by the message
            type: string
        required:
        - topic
        - error
  schemas:
    MyInfo:
      payload:
        title: MyInfo
        type: object
        properties:
          mobile:
            title: Mobile
            example: '+385987654321'
            type: string
          name:
            title: Name
            example: James Bond
            type: string
        required:
        - mobile
        - name
  securitySchemes: {}
"""

In [ ]:
async def test_me():
    async with start_test_app() as (app, test_produce_function):
        client = TestClient(app)
        response = client.get("/asyncapi.yml")
        assert response.status_code == 200
        assert yaml.safe_load(response.text) == yaml.safe_load(
            expected
        ), f"{yaml.safe_load(response.text)} != {yaml.safe_load(expected)}"
        await asyncio.sleep(2)
        
        await test_produce_function(msg=MyMsgUrl(info=dict(mobile="+385987654321", name="James Bond"), url="https://sis.gov.uk/agents/007"))

asyncio.run(test_me())

print("ok")

In [ ]:
# | export


@patch
def produce_raw(
    self: FastKafkaAPI,
    topic: str,
    raw_msg: Union[str, bytes],
    on_delivery: Optional[Callable[[KafkaMessage, Message], None]] = None,
) -> "asyncio.Future[Any]":

    if isinstance(raw_msg, str):
        raw_msg = raw_msg.encode("utf-8")

    if on_delivery is None:
        on_delivery = self._store["producers"][topic]  # type: ignore

    if iscoroutinefunction(on_delivery):
        raise ValueError("coroutines not supported for callbacks yet")

    p: AIOProducer = self._confluent_producer  # type: ignore

    def _delivery_report(
        kafka_err: KafkaError,
        kafka_msg: Message,
        self=self,
        topic=topic,
        raw_msg=raw_msg,
        on_delivery=on_delivery,
    ):
        msg_cls: KafkaMessage
        if kafka_err is not None:
            logger.info(f"produce_raw() topic={topic} raw_msg={raw_msg} delivery error")
            if self._on_error_topic is not None:
                on_error = self._store["producers"][self._on_error_topic]
                msg_cls = _get_msg_cls_for_method(on_error)
                on_error(
                    msg_cls("Message delivery failed: {}".format(kafka_err)), kafka_err  # type: ignore
                )
        else:
            logger.info(f"produce_raw() topic={topic} raw_msg={raw_msg} delivered")
            msg_cls = _get_msg_cls_for_method(on_delivery)
            on_delivery(msg_cls.parse_raw(raw_msg), kafka_msg)

    return p.produce(topic, raw_msg, on_delivery=_delivery_report)

In [ ]:
raw_msg = (
    MyMsgUrl(
        info=dict(mobile="+385987654321", name="James Bond"),
        url="https://sis.gov.uk/agents/007",
    )
    .json()
    .encode("utf-8")
)


async def test_me():
    async with start_test_app() as (app, _):
        await app.produce_raw("my_topic_3", raw_msg)

        def _on_delivery(msg: KafkaMessage, *args):
            logger.warning("me so cool")

        # we don't need to wait for it
        app.produce_raw("my_topic_3", raw_msg, on_delivery=_on_delivery)


asyncio.run(test_me())


print("ok")

In [ ]:
# |export


@patch
def test_run(self: FastKafkaAPI, f: Callable[[], Any], timeout: int = 30):
    async def _loop(app: FastKafkaAPI = self, f: Callable[[], Any] = f):
        logger.info(f"test_run(): starting")
        try:
            async with anyio.create_task_group() as tg:
                with anyio.move_on_after(timeout) as scope:
                    app._on_startup()  # type: ignore

                    if iscoroutinefunction(f):
                        logger.info(f"test_run(app={app}, f={f}): Calling coroutine {f}")
                        retval = await f()
                    else:
                        logger.info(f"test_run(app={app}, f={f}): Calling function {f}")
                        retval = await asyncer.asyncify(f)()

                return retval
        except Exception as e:
            logger.error(f"test_run(): exception caugth {e}")
            raise e
        finally:
            logger.info(f"test_run(app={app}, f={f}): shutting down the app")
            await app._on_shutdown()
            logger.info(f"test_run(app={app}, f={f}): finished")

    return asyncer.runnify(_loop)()

In [ ]:
retval = app.test_run(lambda: print("Hello world"))
print(f"retval={retval},")

In [ ]:
# |export


@patch
@asynccontextmanager
async def testing_ctx(self: FastKafkaAPI, timeout: int = 30):
    logger.info(f"test_context(): starting")
    try:
        async with anyio.create_task_group() as tg:
            with anyio.move_on_after(timeout) as scope:
                self._on_startup()  # type: ignore

                yield

    except Exception as e:
        logger.error(f"test_context(): exception caugth {e}")
        raise e
    finally:
        logger.info(f"test_context(self={self}): shutting down the app")
        await self._on_shutdown()
        logger.info(f"test_context(self={self}): finished")

In [ ]:
async with app.testing_ctx():
    print("app is up and running")
    await anyio.sleep(2)
print("app is shut down")

In [ ]:
async with app.testing_ctx(timeout=3):
    print("app is up and running")
    await anyio.sleep(1000)
print("app is shut down")

In [ ]:
# | eval: false

# uvicorn.run(app, host="0.0.0.0", port=6006)